# IPv4 Addressing

- 32 bits = 4 bytes
- address space = 2^32 addresses
- dotted decimal notation: A.B.C.D
    - A, B, C and D between 0 and 255


In [1]:
# Check if the given string is a valid IPv4 address
def is_valid(ip):
    ip = ip.split('.')
    if len(ip) != 4:
        return False
    for i in ip:
        if not i.isdigit() or not 0 <= int(i) <= 255:
            return False
    return True

In [3]:
print(is_valid('10.20.300.40'))

False


In [3]:
# Convert an IPv4 address to binary
def to_binary(ip):
    ip = ip.split('.')
    return ' '.join([bin(int(i))[2:].zfill(8) for i in ip])

In [4]:
print(to_binary('127.0.0.1'))

01111111 00000000 00000000 00000001


In [7]:
# Convert a binary IP address to decimal
def to_decimal(ip):
    ip = ip.split('.')
    return '.'.join([str(int(i, 2)) for i in ip])

In [8]:
print(to_decimal('01111111.00000000.00000000.00000001'))

127.0.0.1


# IPv4 Classes

| Class            | Binary Prefix | Address Range               | Default Subnet Mask | Networks           | Hosts                 |
|------------------|---------------|-----------------------------|---------------------|--------------------|-----------------------|
| A                | 0             | 1.0.0.0 - 127.255.255.255   | 255.0.0.0 (/8)      | 7 bits - 128 nets  | 24 bits - 16M hosts   |
| B                | 10            | 128.0.0.0 - 191.255.255.255 | 255.255.0.0 (/16)   | 14 bits - 16k nets | 16 bits - 64k hosts   |
| C                | 110           | 192.0.0.0 - 223.255.255.255 | 255.255.255.0 (/24) | 21 bits - 2M nets  | 8 bits - 256 hosts    |
| D (multicast)    | 1110          | 224.0.0.0 - 239.255.255.255 | Not applicable      | Not applicable     | 256M multicast groups |
| E (experimental) | 1111          | 240.0.0.0 - 255.255.255.254 | Not applicable      | Not applicable     | Not applicable        |

In [9]:
# Return IPv4 class
def get_ip_class(ip):
    ip = ip.split('.')
    if 1 <= int(ip[0]) <= 127:
        return 'A'
    elif 128 <= int(ip[0]) <= 191:
        return 'B'
    elif 192 <= int(ip[0]) <= 223:
        return 'C'
    elif 224 <= int(ip[0]) <= 239:
        return 'D'
    elif 240 <= int(ip[0]) <= 255:
        return 'E'

In [10]:
print(get_ip_class('200.10.20.5'))

C


# Classless addressing (CIDR - Classless Inter-Domain Routing)

- IP addresses are allocated more efficiently according to actual need
- VLSM (Variable-Length Subnet Masking) - subnet mask can be of any length
- notation - IP followed by a slash ("/") and a number indicating the number of bits in the network prefix (ex. 192.168.1.0/24)
- simplifies routing - routes can be aggregated into a single routing table entry

In [17]:
# Convert CIDR notation to IPv4 address
def cidr_to_ip(cidr):
    ip, mask = cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    return '.'.join(ip) + ' ' + '.'.join(mask)

# Example using the function
ip = cidr_to_ip('/8')
print(ip, to_binary(ip))

 255.0.0.0 11111111 00000000 00000000 00000000


In [20]:
# Return number of addresses in a subnet given the CIDR notation
def get_num_addr(cidr):
    return 2 ** (32 - int(cidr))

print(get_num_addr(8))

16777216


# Reserved address

- cannot be assigned to any host

- network address - represents the entire subnet
    - network_address = IP address AND subnet mask
    - all host bits with 0
- broadcast address - used to send data to all devices on the network
    - broadcast_address = IP address OR INVERT(subnet mask)
    - all host bits with 1

In [30]:
# Return network address
def get_network_addr(ip_cidr):
    ip, mask = ip_cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    network = [str(int(ip[i]) & int(mask[i])) for i in range(4)]
    return '.'.join(network)

print(get_network_addr('255.255.255.255/1'))


128.0.0.0


In [31]:
# Return broadcast address
def get_broadcast_addr(ip_cidr):
    ip, mask = ip_cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    broadcast = [str(int(ip[i]) | (255 - int(mask[i]))) for i in range(4)]
    return '.'.join(broadcast)

print(get_broadcast_addr('192.168.10.5/16'))

192.168.255.255


In [32]:
# Return first and last usable addresses in a subnet
def get_first_last_addr(ip_cidr):
    ip, mask = ip_cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    network = [str(int(ip[i]) & int(mask[i])) for i in range(4)]
    broadcast = [str(int(ip[i]) | (255 - int(mask[i]))) for i in range(4)]
    first = network.copy()
    first[-1] = str(int(first[-1]) + 1)
    last = broadcast.copy()
    last[-1] = str(int(last[-1]) - 1)
    return '.'.join(first), '.'.join(last)

print(get_first_last_addr('192.168.10.5/16'))

('192.168.0.1', '192.168.255.254')
